# AME project 3


# Racial differences in police use of force

The goal of this project is to investigate whether there are racial differences in police use of force. We will analyse this issue based on two different binary response models: the Probit model and the Logit model.
Binary response models are relevant when the dependent variable $y$ has two possible outcomes, 
e.g., $y=1$ if a police encounter resulted in any use of force by the police officer(s), and $y=0$ if it does not.


## Theory

The binary response model assumes that the data generating process is 
$$
\begin{aligned}
y_i^* &= \mathbf{x}_i \boldsymbol{\beta} + u_i, \\ 
y_i   &= \mathbf{1}(y_i^* > 0), 
\end{aligned}
$$
where $u_i$ are distributed IID according to some cdf. $G$. 

In the lectures, it is shown that
$$ p_i \equiv \Pr(y_i = 1| \mathbf{x}_i) = G(\mathbf{x}_i \boldsymbol{\beta}). $$ 

Since $y_i$ (conditioning on $\mathbf{x}_i$) is Bernoulli-distributed with parameter $p_i$, its log-likelihood function is 
$$
\ell_i(\theta) 
               = \mathrm{1}(y_i = 1) \log[ G(\mathbf{x}_i \boldsymbol{\beta}) ]
               + \mathrm{1}(y_i = 0) \log[1 - G(\mathbf{x}_i \boldsymbol{\beta})]
$$

Estimation is then conducted by maximum likelihood, 
$$ \hat{\boldsymbol{\theta}} = \arg\max_\theta \frac{1}{N} \sum_{i=1}^N \ell_i (\theta), $$ 
which can be implemented as a minimizer in the usual $M$-framework with $q(\theta, y_i, x_i) = -\ell_i(\theta)$, and then minimizing $Q(\theta) = N^{-1} \sum_i q(\theta, y_i, x_i)$. 

We will consider two models: 
1. Probit: when $G$ is the standard normal CDF 
2. Logit: when $G$ is the standard logistic CDF. 

## Setup 

In [10]:
from sys import path

import numpy as np
import pandas as pd 
from scipy.stats import norm
import matplotlib.pyplot as plt 
import seaborn as sns 
sns.set_theme()

%load_ext autoreload
%autoreload 2

import estimation as est 
import probit
import logit

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Inspecting and adjusting data




In [11]:
dat = pd.read_csv('ppcs_cc.csv')
print(f'The data contains {dat.shape[0]} rows (encounters) and {dat.shape[1]} columns (variables).')

The data contains 3799 rows (encounters) and 19 columns (variables).


In [12]:
#Summary statistics for variables  
print(dat.describe())

#Possible values of each variable
print('Possible values of each variable:')
for col in dat.columns:
    print(f'{col}: {dat[col].unique()}')

            sblack        shisp       swhite       sother        smale  \
count  3799.000000  3799.000000  3799.000000  3799.000000  3799.000000   
mean      0.110555     0.101606     0.739142     0.048697     0.529613   
std       0.313622     0.302169     0.439160     0.215262     0.499188   
min       0.000000     0.000000     0.000000     0.000000     0.000000   
25%       0.000000     0.000000     0.000000     0.000000     0.000000   
50%       0.000000     0.000000     1.000000     0.000000     1.000000   
75%       0.000000     0.000000     1.000000     0.000000     1.000000   
max       1.000000     1.000000     1.000000     1.000000     1.000000   

              sage        sempl      sincome         spop      daytime  \
count  3799.000000  3799.000000  3799.000000  3799.000000  3799.000000   
mean     41.010003     0.695446     2.164780     1.362727     0.666491   
std      16.146916     0.460279     0.848262     0.765598     0.471529   
min      16.000000     0.000000     1

In [13]:
#Dimension of the data
N = dat.shape[0]

#Exclude the variables with zero variance
dat = dat.drop(columns=['osplit', 'year'])

#Transform the variables 'sincome', 'spop', 'inctype_lin' to dummy variables
dat = pd.get_dummies(dat, columns=['inctype_lin', 'sincome', 'spop'], drop_first=True, dtype=int)

#Adjust names of new dummies
dat.rename(columns={'inctype_lin_2': 'traffic_stop', 'sincome_2': 'sincome_20-50', 'sincome_3': 'sincome_above50', 'spop_2': 'spop_100-499', 'spop_3':'spop_500-999', 'spop_4': 'spop_above1000' }, inplace=True)

print(f'The data now contains {dat.shape[0]} rows (encounters) and {dat.shape[1]} columns (variables).')

#divide age by 10
dat['sage'] = dat['sage']/10

#Adding constant and squared age
dat['const'] = np.ones((N,))
dat['sagesq'] = dat['sage'] * dat['sage']

#Count the number of times swhite, sblack, shisp, sother is one when anyuseofforce_coded is one
white_force = dat['swhite'] * dat['anyuseofforce_coded']
black_force = dat['sblack'] * dat['anyuseofforce_coded']
hisp_force = dat['shisp'] * dat['anyuseofforce_coded']
other_force = dat['sother'] * dat['anyuseofforce_coded']

white=dat['swhite'].sum()
black=dat['sblack'].sum()
hisp=dat['shisp'].sum()
other=dat['sother'].sum()

#print
print(f'White subjects in data: {white}')
print(f'Force were used against {white_force.sum()} of the white subjects')
print(f'Black subjects in data: {black}')
print(f'Force were used against {black_force.sum()} of the black subjects')
print(f'Hispanic subjects in data: {hisp}')
print(f'Force were used against {hisp_force.sum()} of the Hispanic subjects')
print(f'Other subjects in data: {other}')
print(f'Force were used against {other_force.sum()} of the other subjects')

The data now contains 3799 rows (encounters) and 20 columns (variables).
White subjects in data: 2808
Force were used against 9 of the white subjects
Black subjects in data: 420
Force were used against 3 of the black subjects
Hispanic subjects in data: 386
Force were used against 6 of the Hispanic subjects
Other subjects in data: 185
Force were used against 1 of the other subjects


# Model estimation

In [14]:
# Declare labels
y_lab = 'anyuseofforce_coded' # dependent variable
x_lab = ['const', 
         'sblack', 'shisp','sother', #swhite is the reference category
         'smale',  'sbehavior',
         'sage', 'sagesq',
         'sincome_20-50', 'sincome_above50', #sincome_0-20 is the reference category
         'omajwhite',# the reference category is that the officers belong to a racial minority ('omajblack', 'omajhisp', 'omajother') 
         'daytime', 'traffic_stop'
        ]

# reorder columns 
data = dat[[y_lab] + x_lab].copy()

y = data[y_lab].values
x = data[x_lab].values
K = x.shape[1]

### Probit

In [15]:
beta0_probit = probit.starting_values(y, x)

probit_results = est.estimate(probit.q, beta0_probit, y, x)
probit_tab = est.print_table(x_lab, probit_results, title=f'Probit, y = {y_lab}')

Optimization terminated successfully.
         Current function value: 0.023516
         Iterations: 106
         Function evaluations: 1722
         Gradient evaluations: 123
Optimizer succeded after 106 iter. (1722 func. evals.). Final criterion:  0.02352.
Probit, y = anyuseofforce_coded


### Logit

In [16]:
beta0_logit = logit.starting_values(y, x)

logit_results = est.estimate(logit.q, beta0_logit, y, x)
logit_tab = est.print_table(x_lab, logit_results, title=f'Logit, y = {y_lab}')

Optimization terminated successfully.
         Current function value: 0.023638
         Iterations: 125
         Function evaluations: 1988
         Gradient evaluations: 142
Optimizer succeded after 125 iter. (1988 func. evals.). Final criterion:  0.02364.
Logit, y = anyuseofforce_coded


### Print all results

In [17]:
#Table with results for all models
results = pd.concat([probit_tab, logit_tab], axis=1)

#Adjusting column names
results.columns = ['Probit coef', 'Probit se', 'Probit t', 'Logit coef', 'Logit se', 'Logit t']

results

,Probit coef,Probit se,Probit t,Logit coef,Logit se,Logit t
const,-3.1622,1.9742,-1.6018,-7.0497,4.3546,-1.6189
sblack,0.2928,0.3703,0.7908,0.6686,0.8705,0.7681
shisp,0.5404,0.3100,1.7431,1.3226,0.6750,1.9595
sother,0.2271,0.6589,0.3447,0.3078,1.7987,0.1711
smale,0.4778,0.4020,1.1886,1.0375,0.8933,1.1614
sbehavior,1.0742,0.3917,2.7423,2.5830,0.8447,3.0580
sage,0.2335,0.7541,0.3097,1.0070,1.7503,0.5753
sagesq,-0.0456,0.0945,-0.4822,-0.1826,0.2313,-0.7894
sincome_20-50,0.0582,0.4507,0.1291,0.1949,0.9790,0.1991
sincome_above50,0.0874,0.4037,0.2165,0.1050,0.8497,0.1236


In [18]:
print(pd.DataFrame.to_latex(results, float_format="%.2f"))

\begin{tabular}{lrrrrrr}
\toprule
 & Probit coef & Probit se & Probit t & Logit coef & Logit se & Logit t \\
\midrule
const & -3.16 & 1.97 & -1.60 & -7.05 & 4.35 & -1.62 \\
sblack & 0.29 & 0.37 & 0.79 & 0.67 & 0.87 & 0.77 \\
shisp & 0.54 & 0.31 & 1.74 & 1.32 & 0.68 & 1.96 \\
sother & 0.23 & 0.66 & 0.34 & 0.31 & 1.80 & 0.17 \\
smale & 0.48 & 0.40 & 1.19 & 1.04 & 0.89 & 1.16 \\
sbehavior & 1.07 & 0.39 & 2.74 & 2.58 & 0.84 & 3.06 \\
sage & 0.23 & 0.75 & 0.31 & 1.01 & 1.75 & 0.58 \\
sagesq & -0.05 & 0.09 & -0.48 & -0.18 & 0.23 & -0.79 \\
sincome_20-50 & 0.06 & 0.45 & 0.13 & 0.19 & 0.98 & 0.20 \\
sincome_above50 & 0.09 & 0.40 & 0.22 & 0.10 & 0.85 & 0.12 \\
omajwhite & 0.47 & 0.81 & 0.57 & 0.93 & 1.69 & 0.55 \\
daytime & -0.15 & 0.31 & -0.49 & -0.33 & 0.69 & -0.48 \\
traffic_stop & -0.81 & 0.32 & -2.52 & -1.93 & 0.69 & -2.80 \\
\bottomrule
\end{tabular}



# Significance of racial differences

In the above models we use "swhite" as the baseline race category allowing us to make inference about the difference between being a white subject or a black/hispanic/other subject. We now explore the differences between black and hispanic subjects, black and other subjects, and hispanic and other subjects. 

We base the coefficient differences on the estimates from the above models, and base the standard errors of the differences on the delta method. 

In [19]:
## DELTA METHOD

# Gradient vector
x_black = np.zeros(K)
x_hisp = np.zeros(K)
x_other = np.zeros(K)
x_black[1] = 1
x_hisp[2] = 1
x_other[3] = 1

grad_hisp_black = x_hisp - x_black
grad_other_black = x_other - x_black
grad_other_hisp = x_other - x_hisp

In [20]:
def get_se(grad, cov):
    cov_me = grad@cov@grad.T
    return np.sqrt((cov_me))

In [21]:
#STANDARD ERRORS FOR DIFFERENCES

#Probit
probit_se_hisp_black = get_se(grad_hisp_black, probit_results['cov'])
probit_se_other_black = get_se(grad_other_black, probit_results['cov'])
probit_se_other_hisp = get_se(grad_other_hisp, probit_results['cov'])

#Logit
logit_se_hisp_black = get_se(grad_hisp_black, logit_results['cov'])
logit_se_other_black = get_se(grad_other_black, logit_results['cov'])
logit_se_other_hisp = get_se(grad_other_hisp, logit_results['cov'])

In [22]:
#COEFFICIENTS FOR DIFFERENCES

#Probit
probit_coef_hisp_black = probit_results['beta'][2] - probit_results['beta'][1]
probit_coef_other_black = probit_results['beta'][3] - probit_results['beta'][1]
probit_coef_other_hisp = probit_results['beta'][3] - probit_results['beta'][2]

#Logit
logit_coef_hisp_black = logit_results['beta'][2] - logit_results['beta'][1]
logit_coef_other_black = logit_results['beta'][3] - logit_results['beta'][1]
logit_coef_other_hisp = logit_results['beta'][3] - logit_results['beta'][2]

In [25]:
#Print results in a table
diff_table = pd.DataFrame({'Probit coef': [probit_coef_hisp_black, probit_coef_other_black, probit_coef_other_hisp],
                           'Probit se': [probit_se_hisp_black, probit_se_other_black, probit_se_other_hisp],
                           'Probit t': [probit_coef_hisp_black/probit_se_hisp_black, probit_coef_other_black/probit_se_other_black, probit_coef_other_hisp/probit_se_other_hisp],
                           'Logit coef': [logit_coef_hisp_black, logit_coef_other_black, logit_coef_other_hisp],
                           'Logit se': [logit_se_hisp_black, logit_se_other_black, logit_se_other_hisp],
                           'Logit t': [logit_coef_hisp_black/logit_se_hisp_black, logit_coef_other_black/logit_se_other_black, logit_coef_other_hisp/logit_se_other_hisp]},       
                            index=['shisp-sblack', 'sother-sblack', 'sother-shisp'])

diff_table

,Probit coef,Probit se,Probit t,Logit coef,Logit se,Logit t
shisp-sblack,0.247519,0.467451,0.529507,0.654007,1.002226,0.652555
sother-sblack,-0.065736,0.739911,-0.088844,-0.360828,2.011861,-0.179350
sother-shisp,-0.313255,0.731425,-0.428280,-1.014835,1.903583,-0.533119


In [26]:
print(pd.DataFrame.to_latex(diff_table, float_format="%.2f"))

\begin{tabular}{lrrrrrr}
\toprule
 & Probit coef & Probit se & Probit t & Logit coef & Logit se & Logit t \\
\midrule
shisp-sblack & 0.25 & 0.47 & 0.53 & 0.65 & 1.00 & 0.65 \\
sother-sblack & -0.07 & 0.74 & -0.09 & -0.36 & 2.01 & -0.18 \\
sother-shisp & -0.31 & 0.73 & -0.43 & -1.01 & 1.90 & -0.53 \\
\bottomrule
\end{tabular}



# Average Partial Effect (APE)

Above we only found slightly significant racial differences in the probability of policy use of force between white and hispanic subjects. Here we calculate the actual magnitude of this difference.

In [27]:
#Creating two new datasets - one where all respondents are coded as white and one where all respondents are coded as Hispanic
x_swhite = x.copy()
x_swhite[:, x_lab.index('shisp')] = 0
x_swhite[:, x_lab.index('sblack')] = 0
x_swhite[:, x_lab.index('sother')] = 0

x_shisp = x.copy()
x_shisp[:, x_lab.index('shisp')] = 1
x_shisp[:, x_lab.index('sblack')] = 0
x_shisp[:, x_lab.index('sother')] = 0

### APE based on probit results

In [28]:
PE_probit=probit.G(x_shisp @ probit_results['beta'])-probit.G(x_swhite @ probit_results['beta'])
APE_probit=PE_probit.mean()
print(f'Average Partial Effect of being Hispanic rather than white: {APE_probit:.4f}')
print(f'I.e the probability of the police using force is, on average, {APE_probit*100:.2f}% higher if the subject is Hispanic rather than white.')

Average Partial Effect of being Hispanic rather than white: 0.0083
I.e the probability of the police using force is, on average, 0.83% higher if the subject is Hispanic rather than white.


### APE based on logit results

In [29]:
PE_logit=logit.G(x_shisp @ logit_results['beta'])-logit.G(x_swhite @ logit_results['beta'])
APE_logit=PE_logit.mean()
print(f'Average Partial Effect of being Hispanic rather than white: {APE_logit:.4f}')
print(f'I.e the probability of the police using force is, on average, {APE_logit*100:.2f}% higher if the subject is Hispanic rather than white.')

Average Partial Effect of being Hispanic rather than white: 0.0083
I.e the probability of the police using force is, on average, 0.83% higher if the subject is Hispanic rather than white.


In [31]:
# print results 
APE_table = pd.DataFrame([APE_probit,
                APE_logit],
             index=['Probit', 'Logit'], columns=[f'APE']).round(4)
APE_table

,APE
Probit,0.0083
Logit,0.0083


In [32]:
print(pd.DataFrame.to_latex(APE_table, float_format="%.4f"))

\begin{tabular}{lr}
\toprule
 & APE \\
\midrule
Probit & 0.0083 \\
Logit & 0.0083 \\
\bottomrule
\end{tabular}

